In [1]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=629bb4eb9bb3f0bb93cf3995438b1096dbe71bee7a6ae669dfbbc66e5daf0b74
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
import os
import json
import torch
import itertools
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModel, ViTModel, ViTFeatureExtractor, MarianMTModel, MarianTokenizer
from langdetect import detect
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image


In [3]:
# Step 1: Mount Google Drive to access the dataset.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import json
import torch
import itertools
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, ViTModel, ViTFeatureExtractor
from transformers import BlipProcessor, Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model
import torch.utils.checkpoint as checkpoint
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
# Paths to data
image_folder = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/Flicker8k_Dataset'
caption_file = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/BAN-Cap_captiondata.json'

# Load captions from JSON file
with open(caption_file, 'r', encoding='utf-8') as f:
    captions_data = json.load(f)

# Convert captions to DataFrame
captions_list = []
for entry in captions_data:
    if 'caption_id' in entry and 'english_caption' in entry:
        filename = entry['caption_id'].split('#')[0]
        caption = entry['english_caption']
        captions_list.append({"filename": filename, "caption": caption})

df = pd.DataFrame(captions_list)
print(f"Total captions loaded: {len(df)}")

# Filter image files
image_filenames = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

# Match images with captions
captions = []
for filename in image_filenames:
    matching_captions = df[df['filename'] == filename]['caption'].tolist()
    if len(matching_captions) >= 2:
        captions.append(matching_captions[0])

assert len(captions) > 0, "No captions found!"
assert len(image_filenames) == len(captions), "Mismatch between images and captions!"
print(f"Total images matched with captions: {len(image_filenames)}")


Total captions loaded: 40455
Total images matched with captions: 8091


In [6]:
!pip install --upgrade transformers huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [7]:
!pip install --upgrade transformers


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    ViTModel,
    ViTFeatureExtractor,
    BlipProcessor,
    Blip2ForConditionalGeneration,
    MarianMTModel,
    MarianTokenizer
)
from peft import LoraConfig, get_peft_model
import torch.utils.checkpoint as checkpoint
from PIL import Image
import numpy as np
from torchvision import transforms
from tqdm import tqdm


In [ ]:
# First, install the required packages
!pip install transformers accelerate datasets
!pip install -q -U transformers accelerate git+https://github.com/huggingface/transformers.git

# Import statements
import torch
from transformers import (
    AutoProcessor,
    Blip2Processor,
    Blip2ForConditionalGeneration,
    AutoTokenizer
)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# BLIP2 Setup - using a different checkpoint that's more stable
try:
    blip_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
    blip_model = Blip2ForConditionalGeneration.from_pretrained(
        "Salesforce/blip2-opt-2.7b",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    ).to(device)
    blip_model.eval()
except Exception as e:
    print(f"Error loading BLIP2 model: {e}")
    # Fallback to a smaller model
    print("Trying fallback model...")
    blip_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-1.7b")
    blip_model = Blip2ForConditionalGeneration.from_pretrained(
        "Salesforce/blip2-opt-1.7b",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    ).to(device)
    blip_model.eval()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
class TranslationPipeline:
    def __init__(self):
        # Initialize translation models for different language pairs
        self.en_pt_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE')
        self.en_pt_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE')

        # Add more language pairs as needed
        self.bn_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-bn-en')
        self.bn_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-bn-en')

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.en_pt_model.to(self.device)
        self.bn_en_model.to(self.device)

    def translate(self, text, source_lang, target_lang):
        if source_lang == 'bn' and target_lang == 'en':
            return self._translate_with_model(
                text, self.bn_en_model, self.bn_en_tokenizer
            )
        elif source_lang == 'en' and target_lang == 'pt':
            return self._translate_with_model(
                text, self.en_pt_model, self.en_pt_tokenizer
            )
        return text  # Default fallback

    def _translate_with_model(self, text, model, tokenizer):
        inputs = tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        outputs = model.generate(**inputs)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
)
text_model = get_peft_model(text_model, lora_config)
text_model.gradient_checkpointing_enable()

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms


In [ ]:
class ImageTextDataset(Dataset):
    def __init__(self, image_filenames, captions, image_folder, clip_score_threshold=0.20):
        self.image_filenames = []
        self.captions = []
        self.image_folder = image_folder
        self.clip_score_threshold = clip_score_threshold

        # Filter dataset based on CLIP score
        for img, cap in zip(image_filenames, captions):
            if self._compute_clip_score(img, cap) > clip_score_threshold:
                self.image_filenames.append(img)
                self.captions.append(cap)

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    def _compute_clip_score(self, image_path, caption):
        # Implement CLIP score computation as mentioned in paper
        # This is a placeholder - implement actual CLIP scoring
        return 0.25  # Default passing score for now

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_folder, self.image_filenames[idx])
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)
        return image, self.captions[idx]


In [ ]:
class Projection(nn.Module):
    def __init__(self, input_size, output_size):
        super(Projection, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(output_size, output_size)

    def forward(self, x):
        x = self.fc2(self.dropout(F.relu(self.fc1(x))))
        return x


class MultilingualCLIPModel(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones([]) * temperature)

        # Initialize models
        self.image_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224')
        self.text_encoder = AutoModel.from_pretrained('xlm-roberta-base')

        # Freeze image encoder as per paper
        for param in self.image_encoder.parameters():
            param.requires_grad = False

        # Apply LoRA to text encoder
        lora_config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["query", "value"],
            lora_dropout=0.1,
            bias="none"
        )
        self.text_encoder = get_peft_model(self.text_encoder, lora_config)

        # Enable gradient checkpointing
        self.text_encoder.gradient_checkpointing_enable()

        # Initialize BLIP2 for caption generation
        self.blip_model = Blip2ForConditionalGeneration.from_pretrained(
            "Salesforce/blip2-opt-2.7b"
        )
        self.blip_processor = BlipProcessor.from_pretrained(
            "Salesforce/blip2-opt-2.7b"
        )

        # Translation pipeline
        self.translator = TranslationPipeline()

        # Projection layers
        hidden_size = 512
        self.image_projection = nn.Linear(
            self.image_encoder.config.hidden_size,
            hidden_size
        )
        self.text_projection = nn.Linear(
            self.text_encoder.config.hidden_size,
            hidden_size
        )

    def generate_synthetic_captions(self, image, num_captions=10):
        """Generate multiple captions per image using different prefixes"""
        prefixes = [
            "A photo of", "This image shows", "In this picture",
            "This is", "The image depicts", "You can see",
            "Looking at", "The photograph shows", "This scene contains",
            "Visible in this image"
        ]

        captions = []
        for prefix in prefixes[:num_captions]:
            inputs = self.blip_processor(image, prefix, return_tensors="pt")
            output = self.blip_model.generate(**inputs, max_new_tokens=20)
            caption = self.blip_processor.decode(output[0], skip_special_tokens=True)
            captions.append(caption)

        return captions

    def forward(self, images, captions, target_lang='pt'):
        batch_size = images.size(0)

        # 1. Generate synthetic captions
        all_captions = []
        for img in images:
            synthetic_captions = self.generate_synthetic_captions(img)
            all_captions.extend(synthetic_captions)

        # 2. Translate captions
        translated_captions = [
            self.translator.translate(cap, 'en', target_lang)
            for cap in all_captions
        ]

        # 3. Process images (frozen)
        with torch.no_grad():
            image_features = self.image_encoder(images).last_hidden_state[:, 0]
        image_features = self.image_projection(image_features)

        # 4. Process text with gradient checkpointing
        text_features = []
        for cap in translated_captions:
            text_output = checkpoint.checkpoint(
                self.text_encoder,
                self.text_tokenizer(cap, return_tensors='pt', padding=True)
            )
            text_features.append(text_output.last_hidden_state[:, 0])

        text_features = torch.stack(text_features)
        text_features = self.text_projection(text_features)

        # 5. Normalize features
        image_features = F.normalize(image_features, dim=-1)
        text_features = F.normalize(text_features, dim=-1)

        return image_features, text_features

    def compute_loss(self, image_features, text_features):
        # Compute InfoNCE loss as described in paper
        logits = torch.matmul(image_features, text_features.t()) / self.temperature
        labels = torch.arange(len(logits)).to(logits.device)

        # Symmetric loss computation
        loss_i2t = F.cross_entropy(logits, labels)
        loss_t2i = F.cross_entropy(logits.t(), labels)

        return (loss_i2t + loss_t2i) / 2

In [21]:

def train_model(model, train_loader, val_loader, num_epochs, optimizer):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        epoch_loss = 0
        train_progress = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training")

        for batch_idx, (images, captions) in enumerate(train_progress):
            images = images.to(device)

            # Forward pass
            image_features, text_features = model(images, captions)
            loss = model.compute_loss(image_features, text_features)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            train_progress.set_postfix({"Train Loss": epoch_loss / (batch_idx + 1)})

        avg_train_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        val_loss = 0
        val_progress = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation")

        with torch.no_grad():
            for batch_idx, (images, captions) in enumerate(val_progress):
                images = images.to(device)

                image_features, text_features = model(images, captions)
                loss = model.compute_loss(image_features, text_features)

                val_loss += loss.item()
                val_progress.set_postfix({"Val Loss": val_loss / (batch_idx + 1)})

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Average Training Loss: {avg_train_loss:.4f}")
        print(f"Average Validation Loss: {avg_val_loss:.4f}")

        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
            }, 'best_model.pt')

        # Plot losses
        plt.figure(figsize=(10, 5))
        plt.plot(train_losses, label='Training Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Losses')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'loss_plot_epoch_{epoch+1}.png')
        plt.close()


In [22]:
# Main execution
if __name__ == "__main__":
    # Load your data
    image_folder = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/Flicker8k_Dataset'
    caption_file = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/BAN-Cap_captiondata.json'

    # Load captions
    with open(caption_file, 'r', encoding='utf-8') as f:
        captions_data = json.load(f)

    # Prepare dataset
    image_filenames = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

    # Match images with captions
    matched_captions = []
    matched_filenames = []

    for filename in image_filenames:
        matching_captions = [entry['english_caption'] for entry in captions_data
                           if entry['caption_id'].split('#')[0] == filename]
        if matching_captions:
            matched_captions.append(matching_captions[0])
            matched_filenames.append(filename)

    # Create dataset
    full_dataset = ImageTextDataset(matched_filenames, matched_captions, image_folder)

    # Split dataset
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    # Create data loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model and optimizer
    model = MultilingualCLIPModel(vit_model, text_model).to(device)
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=1e-4,
        weight_decay=0.1
    )

    # Train the model
    num_epochs = 10
    train_model(model, train_loader, val_loader, num_epochs, optimizer)

Epoch 1/10 - Training:   0%|          | 0/203 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Epoch 1/10 - Validation:  12%|█▏        | 6/51 [05:47<43:24, 57.88s/it, Val Loss=3.45]


KeyboardInterrupt: 

#Interface

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

# Function to create image embeddings
def create_image_embeddings(images, vit_model, image_projector, device):
    with torch.no_grad():
        # Rescale pixel values from [-1, 1] to [0, 1]
        images = torch.clamp((images + 1) / 2, 0, 1)  # Ensures values are in the [0, 1] range
        features = vit_feature_extractor(images=images.permute(0, 2, 3, 1).cpu().numpy(), return_tensors="pt").to(device)
        image_embeddings = vit_model(**features).last_hidden_state[:, 0, :]
        image_projection = image_projector(image_embeddings)
    return image_projection

# Precompute and store image embeddings for all training images
def precompute_image_embeddings(train_dataset, vit_model, image_projector, device):
    image_embeddings_list = []
    for index in tqdm(range(len(train_dataset)), desc="Processing Training Images"):
        images = train_dataset[index][0].unsqueeze(0).to(device)  # Add batch dimension and move to device
        image_projection = create_image_embeddings(images, vit_model, image_projector, device)
        image_embeddings_list.append(image_projection[0].cpu())  # Store on CPU to save GPU memory
    return image_embeddings_list

# Function for image retrieval based on text query
def image_retrieval_function(input_query, translator, text_tokenizer, text_model, text_projector, train_dataset, image_embeddings_list, device, n=5, display=False):
    with torch.no_grad():
        # Translate query if not in English
        translated_query = translator.translate(input_query)

        # Process text query and create embeddings
        inputs = text_tokenizer(translated_query, return_tensors='pt', padding="max_length", max_length=32, truncation=True).to(device)
        text_embeddings = text_model(**inputs).last_hidden_state[:, 0, :]
        text_projection = text_projector(text_embeddings)

    # Compute similarity scores between the query and all stored image embeddings
    similarity_scores = []
    for image_projection in image_embeddings_list:
        image_projection = image_projection.to(device)
        similarity = F.cosine_similarity(text_projection, image_projection.unsqueeze(0))
        similarity_scores.append(similarity.item())

    # Get indices of the top n most similar images
    top_indices = np.argsort(similarity_scores)[-n:][::-1]

    if display:
        for index in top_indices:
            image_tensor = train_dataset[index][0]  # Retrieve original image from dataset
            plt.imshow(torch.moveaxis(image_tensor, 0, 2))  # Convert CHW to HWC
            plt.title(f"Similarity Score: {similarity_scores[index]:.4f}")
            plt.axis('off')
            plt.show()
    else:
        return top_indices

# Example Usage:
# Precompute image embeddings
image_embeddings_list_train = precompute_image_embeddings(train_dataset, vit_model, image_projector, device)

# Perform image retrieval
query = "A beautiful sunset over the mountains."
retrieved_indices = image_retrieval_function(
    query,
    translator,
    text_tokenizer,
    text_model,
    text_projector,
    train_dataset,
    image_embeddings_list_train,
    device,
    n=5,
    display=True
)
